# MIMIC-III data for Acute Kidney Injury (AKI)

Extracting the data points relevant to AKI

In [29]:
# Import libraries
import pandas as pd
import os
import psycopg2
import getpass

In [30]:
# Path to save the extracted values
export_dir = '../extracted_data'
if not os.path.isdir(export_dir):
    os.mkdir(export_dir)

In [31]:
# Create a database connection
user = 'dhruv.sharma'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'

## Loading the data

In [32]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

Password:········


## Data points with urine output in the OUTPUTEVENTS

In [33]:
# Get all outputs related to Urine output with patient
###(43348,43355,43365,43372,43373,43380,44706,43856,43589,43811,43812,43333,43347,43638,43654,43519,
### 43537,43171,43173,43966,43374,43379,43431,3522,45304,43576,43633,43053)
query = \
"""
with dat as 
(
    select oe.subject_id, count(*)
    from outputevents oe
    inner join d_items d on oe.itemid = d.itemid
    where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
    227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
    group by oe.subject_id
)
select dat.subject_id
from dat;
"""

data = pd.read_sql_query(query,con)
data.head()

,subject_id
0,3
1,4
2,6
3,8
4,9


In [53]:
# Get all outputs related to Urine output for each patient
query = \
"""
with ids as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select dat.subject_id
    from dat
),
data as
(
    select le.subject_id, le.hadm_id, le.icustay_id, le.itemid, d.label, le.charttime, le.value, le.valueuom
    from outputevents le
    inner join d_items d on le.itemid = d.itemid
    where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
    227510,226561,227489,226584,226563,226564,226565,226557,226558) and le.valueuom is not NULL
    order by le.subject_id,le.hadm_id, le.icustay_id, le.charttime
)
select data.subject_id, data.hadm_id, data.icustay_id, extract(epoch from data.charttime) as charttime, data.itemid, data.label,
data.value, data.valueuom
from data
left join ids
on data.subject_id = ids.subject_id
order by data.subject_id, data.hadm_id, data.icustay_id, charttime;
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'urine_output.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,itemid,label,value,valueuom
0,3,145834.0,211552.0,4.159300e+09,40055,Urine Out Foley,20.0,ml
1,3,145834.0,211552.0,4.159307e+09,40055,Urine Out Foley,60.0,ml
2,3,145834.0,211552.0,4.159314e+09,40055,Urine Out Foley,5.0,ml
3,3,145834.0,211552.0,4.159321e+09,40055,Urine Out Foley,15.0,ml
4,3,145834.0,211552.0,4.159325e+09,40055,Urine Out Foley,55.0,ml
5,3,145834.0,211552.0,4.159328e+09,40055,Urine Out Foley,42.0,ml
6,3,145834.0,211552.0,4.159339e+09,40055,Urine Out Foley,35.0,ml
7,3,145834.0,211552.0,4.159343e+09,40055,Urine Out Foley,60.0,ml
8,3,145834.0,211552.0,4.159346e+09,40055,Urine Out Foley,30.0,ml
9,3,145834.0,211552.0,4.159350e+09,40055,Urine Out Foley,45.0,ml


## Extracting vital signs for the patients with the urine output

### Data source : CareVue

In [54]:
# ITEMIDs of interest: (52,211,618,676)
query = \
"""
with data_cv as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime, ce.itemid, ce.value
    from chartevents ce
    left join dat
    on ce.subject_id = dat.subject_id
    where ce.itemid in (52,211,618,676)
    order by ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime, ce.itemid
)
select data_cv.subject_id, data_cv.hadm_id, data_cv.icustay_id, extract(epoch from data_cv.charttime) as charttime, d.itemid, d.label, 
data_cv.value
from data_cv
inner join d_items d
on data_cv.itemid = d.itemid
order by data_cv.subject_id, data_cv.hadm_id, data_cv.icustay_id, charttime, d.label;
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'vital_cv.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,itemid,label,value
0,2,163353,243653.0,5.318684e+09,211,Heart Rate,148
1,2,163353,243653.0,5.318685e+09,211,Heart Rate,131
2,2,163353,243653.0,5.318687e+09,211,Heart Rate,144
3,2,163353,243653.0,5.318690e+09,211,Heart Rate,140
4,3,145834,211552.0,4.159277e+09,211,Heart Rate,95
5,3,145834,211552.0,4.159277e+09,618,Respiratory Rate,16
6,3,145834,211552.0,4.159279e+09,52,Arterial BP Mean,259
7,3,145834,211552.0,4.159280e+09,211,Heart Rate,151
8,3,145834,211552.0,4.159281e+09,211,Heart Rate,135
9,3,145834,211552.0,4.159282e+09,52,Arterial BP Mean,60


### Data source : MetaVision

In [55]:
# ITEMIDs of interest: (220052,220045,220210,223761)
query = \
"""
with data_cv as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime, ce.itemid, ce.value
    from chartevents ce
    left join dat
    on ce.subject_id = dat.subject_id
    where ce.itemid in (220052,220045,220210,223761)
    order by ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime, ce.itemid
)
select data_cv.subject_id, data_cv.hadm_id, data_cv.icustay_id, extract(epoch from data_cv.charttime) as charttime , d.itemid, d.label, 
data_cv.value
from data_cv
inner join d_items d
on data_cv.itemid = d.itemid
order by data_cv.subject_id, data_cv.hadm_id, data_cv.icustay_id, charttime, d.label;
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'vital_mv.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,itemid,label,value
0,23,124321,234044.0,5.926565e+09,223761,Temperature Fahrenheit,95.8
1,23,124321,234044.0,5.926565e+09,220045,Heart Rate,77
2,23,124321,234044.0,5.926565e+09,220210,Respiratory Rate,14
3,23,124321,234044.0,5.926565e+09,220045,Heart Rate,75
4,23,124321,234044.0,5.926568e+09,220052,Arterial Blood Pressure mean,86
5,23,124321,234044.0,5.926568e+09,220045,Heart Rate,84
6,23,124321,234044.0,5.926568e+09,220210,Respiratory Rate,18
7,23,124321,234044.0,5.926572e+09,220052,Arterial Blood Pressure mean,86
8,23,124321,234044.0,5.926572e+09,220045,Heart Rate,78
9,23,124321,234044.0,5.926572e+09,220210,Respiratory Rate,11


## Extracting lab measurements for the patients with the urine output

In [57]:
# ITEMIDs of interest: (51464,51006,50912,51478,51076,51480,51097,50813,51088,51100,50818,50821,51491,51265,51002,51003,51300)
query = \
"""
with data as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select le.subject_id, le.hadm_id, le.charttime, le.itemid, le.value, le.valueuom
    from labevents le
    left join dat
    on le.subject_id = dat.subject_id
    where le.itemid in (51464,51006,50912,51478,50882,51480,50822,50813,50960,50824,50818,50821,51491,51265,51002,51003,51300)
    order by le.subject_id, le.hadm_id, le.charttime,le.itemid
)
select data.subject_id, data.hadm_id, extract(epoch from data.charttime) as charttime, d.itemid, d.label, data.value, data.valueuom
from data
inner join d_labitems d
on data.itemid = d.itemid
order by data.subject_id, data.hadm_id, charttime, d.label;
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'lab_vals.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,charttime,itemid,label,value,valueuom
0,2,163353.0,5.318686e+09,51265,Platelet Count,5,K/uL
1,2,163353.0,5.318687e+09,51265,Platelet Count,302,K/uL
2,3,145834.0,4.159270e+09,50882,Bicarbonate,25,mEq/L
3,3,145834.0,4.159270e+09,50912,Creatinine,3.2,mg/dL
4,3,145834.0,4.159270e+09,50960,Magnesium,2.4,mg/dL
5,3,145834.0,4.159270e+09,51265,Platelet Count,282,K/uL
6,3,145834.0,4.159270e+09,51002,Troponin I,<0.3,ng/ml
7,3,145834.0,4.159270e+09,51006,Urea Nitrogen,53,mg/dL
8,3,145834.0,4.159274e+09,51464,Bilirubin,NEG,EU/dL
9,3,145834.0,4.159274e+09,51478,Glucose,NEG,mg/dL


## Extracting ICD9 codes

In [38]:
# ITEMIDs of interest: ('4280','4279','3969','4150','41511','41512','41513','41519','4160','4161','4162','4168','4169',
#                       '4170','4171','4178','4179','4439','4019','3449','3499','2449','586','042','1991','7140','2869',
#                       '27800','2769','2809','2989','311')

query = \
"""
with data as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select diag.subject_id, diag.hadm_id, diag.seq_num, diag.icd9_code
    from diagnoses_icd diag
    left join dat
    on diag.subject_id = dat.subject_id
    where diag.icd9_code in ('4280','4279','3969','4150','41511','41512','41513','41519','4160','4161','4162','4168','4169',
                             '4170','4171','4178','4179','4439','4019','3449','3499','2449','586','042','1991','7140','2869',
                             '27800','2769','2809','2989','311',
                             '490', '4910', '4911','49120','49121','49122','4918','4919','4920','4928','49300','49301',
                             '49301','49302','49310','49311','49312','49320','49321','49322','49381','49382','49390',
                             '49391','49392','4940','4941','4950','4951','4952','4953','4954','4955','4956','4957','4958',
                             '4959','496',          -- chronic pulmoary disease
                             '25000','25001','25002','25003','25010','25011','25012','25013','25020','25021','25022','25023',
                             '25030','25031','25032','25033','25040','25041','25042','25043','25050','25051','25052','25053',
                             '25060','25061','25062','25063','25070','25071','25072','25073','25080','25081','25082','25083',
                             '25090','25091','25092','25093',     -- diabetes
                             '5710','5711','5712','5713','57140','57141','57142','57149','5715','5716','5718','5719', -- liver diseases
                             '53300','53301','53310','53311','53320','53321','53330','53331','53340','53341','53350','53351',
                             '53360','53361','53370','53371','53390','53391',      -- peptic ulcers
                             '20280','20281','20282','20283','20284','20285','20286','20287','20288', -- lymphoma
                             '30500','30501','30502','30503',        -- alcohol abuse
                             '30590','30591','30592','30593')         -- drug abuse
    order by diag.subject_id, diag.hadm_id, diag.seq_num
)
select data.subject_id, data.hadm_id, data.seq_num, data.icd9_code, d.short_title
from data
inner join d_icd_diagnoses d
on data.icd9_code = d.icd9_code
order by data.subject_id, data.hadm_id, data.seq_num, data.icd9_code;
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'comorbidities.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,seq_num,icd9_code,short_title
0,3,145834,6,4280,CHF NOS
1,4,185777,1,042,Human immuno virus dis
2,4,185777,6,5715,Cirrhosis of liver NOS
3,9,150750,3,4280,CHF NOS
4,9,150750,6,4019,Hypertension NOS
5,12,112213,6,4019,Hypertension NOS
6,13,143045,3,25000,DMII wo cmp nt st uncntr
7,13,143045,4,4019,Hypertension NOS
8,17,161087,5,311,Depressive disorder NEC
9,18,188822,1,25080,DMII oth nt st uncntrld


## Mechanical Ventilation

Adapted from George's repo

In [46]:
query = """

with dat as 
(
    select oe.subject_id, count(*)
    from outputevents oe
    inner join d_items d on oe.itemid = d.itemid
    where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
    227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
    group by oe.subject_id
)
select
    ce.subject_id, hadm_id, icustay_id, extract(epoch from charttime) as charttime    -- case statement determining whether it is an instance of mech vent
    , max(
      case
        when itemid is null or value is null then 0 -- can't have null values
        when itemid = 720 and value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
        when itemid = 467 and value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
        when itemid in
          (
          445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
          , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
          , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
          , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
          , 543 -- PlateauPressure
          , 5865,5866,224707,224709,224705,224706 -- APRV pressure
          , 60,437,505,506,686,220339,224700 -- PEEP
          , 3459 -- high pressure relief
          , 501,502,503,224702 -- PCV
          , 223,667,668,669,670,671,672 -- TCPCV
          , 157,158,1852,3398,3399,3400,3401,3402,3403,3404,8382,227809,227810 -- ETT
          , 224701 -- PSVlevel
          )
          THEN 1
        else 0
      end
      ) as MechVent

  from mimiciii.chartevents ce
  left join dat
  on ce.subject_id = dat.subject_id
  where value is not null
  and itemid in
  (
        720 -- vent type
      , 467 -- O2 delivery device
      , 445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
      , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
      , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
      , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
      , 543 -- PlateauPressure
      , 5865,5866,224707,224709,224705,224706 -- APRV pressure
      , 60,437,505,506,686,220339,224700 -- PEEP
      , 3459 -- high pressure relief
      , 501,502,503,224702 -- PCV
      , 223,667,668,669,670,671,672 -- TCPCV
      , 157,158,1852,3398,3399,3400,3401,3402,3403,3404,8382,227809,227810 -- ETT
      , 224701 -- PSVlevel
  )
  group by ce.subject_id, hadm_id, icustay_id, charttime


"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'mech_vent.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,mechvent
0,3,145834,211552.0,4.159282e+09,1
1,3,145834,211552.0,4.159293e+09,1
2,3,145834,211552.0,4.159303e+09,1
3,3,145834,211552.0,4.159307e+09,1
4,3,145834,211552.0,4.159311e+09,1
5,3,145834,211552.0,4.159321e+09,1
6,3,145834,211552.0,4.159323e+09,1
7,3,145834,211552.0,4.159336e+09,1
8,3,145834,211552.0,4.159350e+09,1
9,3,145834,211552.0,4.159354e+09,1


## Vasopressor values

Adapted from Geroge's repo

### data source: CareVue

In [47]:
#extract vasopressor from carevue
query = """
with data as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select inp.subject_id, hadm_id, icustay_id, extract(epoch from charttime) as charttime, itemid,  -- rate, -- rateuom,
    case when itemid in (30120,221906,30047) and rateuom='mcgkgmin' then round(cast(rate as numeric),3) -- norad
    when itemid in (30120,221906,30047) and rateuom='mcgmin' then round(cast(rate/80 as numeric),3)  -- norad
    when itemid in (30119,221289) and rateuom='mcgkgmin' then round(cast(rate as numeric),3) -- epi
    when itemid in (30119,221289) and rateuom='mcgmin' then round(cast(rate/80 as numeric),3) -- epi
    when itemid in (30051,222315) and rate > 0.2 then round(cast(rate*5/60  as numeric),3) -- vasopressin, in U/h
    when itemid in (30051,222315) and rateuom='Umin' and rate < 0.2 then round(cast(rate*5  as numeric),3) -- vasopressin
    when itemid in (30051,222315) and rateuom='Uhr' then round(cast(rate*5/60  as numeric),3) -- vasopressin
    when itemid in (30128,221749,30127) and rateuom='mcgkgmin' then round(cast(rate*0.45  as numeric),3) -- phenyl
    when itemid in (30128,221749,30127) and rateuom='mcgmin' then round(cast(rate*0.45 / 80  as numeric),3) -- phenyl
    when itemid in (221662,30043,30307) and rateuom='mcgkgmin' then round(cast(rate*0.01   as numeric),3) -- dopa
    when itemid in (221662,30043,30307) and rateuom='mcgmin' then round(cast(rate*0.01/80  as numeric),3) else null 
    end as rate_std-- dopa
    -- case when rateuom='mcgkgmin' then 1 when rateuom='mcgmin' then 2 end as uom
    from mimiciii.inputevents_cv inp
    left join dat
    on inp.subject_id = dat.subject_id
    where itemid in (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) and rate is not null
    order by inp.subject_id, hadm_id, icustay_id, charttime, itemid 
)
select data.subject_id, hadm_id, icustay_id, charttime, data.itemid, d.label, rate_std
from data
inner join d_items d
on data.itemid = d.itemid
order by subject_id, hadm_id, icustay_id, charttime, data.itemid
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'vaso_cv.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,itemid,label,rate_std
0,3,145834.0,211552.0,4.159285e+09,30043,Dopamine,0.200
1,3,145834.0,211552.0,4.159285e+09,30128,Neosynephrine-k,0.277
2,3,145834.0,211552.0,4.159287e+09,30043,Dopamine,0.100
3,3,145834.0,211552.0,4.159289e+09,30043,Dopamine,0.040
4,3,145834.0,211552.0,4.159289e+09,30128,Neosynephrine-k,0.279
5,3,145834.0,211552.0,4.159292e+09,30128,Neosynephrine-k,0.279
6,3,145834.0,211552.0,4.159296e+09,30120,Levophed-k,0.103
7,3,145834.0,211552.0,4.159296e+09,30128,Neosynephrine-k,0.279
8,3,145834.0,211552.0,4.159300e+09,30120,Levophed-k,0.103
9,3,145834.0,211552.0,4.159300e+09,30128,Neosynephrine-k,0.279


### data source: MetaVision

In [48]:
#extract vasopressor from metavision
query = """
with data as
(
    with dat as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    
    select inp.subject_id, hadm_id, icustay_id, extract(epoch from starttime) as starttime, extract(epoch from endtime) as endtime, itemid, -- rate, -- ,rateuom,
    case when itemid in (30120,221906,30047) and rateuom='mcg/kg/min' then round(cast(rate as numeric),3)  -- norad
    when itemid in (30120,221906,30047) and rateuom='mcg/min' then round(cast(rate/80 as numeric),3)  -- norad
    when itemid in (30119,221289) and rateuom='mcg/kg/min' then round(cast(rate as numeric),3) -- epi
    when itemid in (30119,221289) and rateuom='mcg/min' then round(cast(rate/80 as numeric),3) -- epi
    when itemid in (30051,222315) and rate > 0.2 then round(cast(rate*5/60  as numeric),3) -- vasopressin, in U/h
    when itemid in (30051,222315) and rateuom='units/min' then round(cast(rate*5 as numeric),3) -- vasopressin
    when itemid in (30051,222315) and rateuom='units/hour' then round(cast(rate*5/60 as numeric),3) -- vasopressin
    when itemid in (30128,221749,30127) and rateuom='mcg/kg/min' then round(cast(rate*0.45 as numeric),3) -- phenyl
    when itemid in (30128,221749,30127) and rateuom='mcg/min' then round(cast(rate*0.45 / 80 as numeric),3) -- phenyl
    when itemid in (221662,30043,30307) and rateuom='mcg/kg/min' then round(cast(rate*0.01 as numeric),3)  -- dopa
    when itemid in (221662,30043,30307) and rateuom='mcg/min' then round(cast(rate*0.01/80 as numeric),3) else null end as rate_std-- dopa
    from mimiciii.inputevents_mv inp
    left join dat
    on inp.subject_id = dat.subject_id
    where itemid in (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) and rate is not null and statusdescription <> 'Rewritten'
    order by inp.subject_id, hadm_id, icustay_id, starttime, itemid    
    
)
select data.subject_id, hadm_id, icustay_id, starttime, data.itemid, d.label, rate_std
from data
inner join d_items d
on data.itemid = d.itemid
order by subject_id, hadm_id, icustay_id, starttime, data.itemid
"""

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'vaso_mv.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,hadm_id,icustay_id,starttime,itemid,label,rate_std
0,111,155897,249202.0,5.506658e+09,221749,Phenylephrine,4.441
1,111,155897,249202.0,5.506658e+09,221906,Norepinephrine,0.500
2,111,155897,249202.0,5.506658e+09,222315,Vasopressin,0.200
3,111,155897,249202.0,5.506659e+09,222315,Vasopressin,0.167
4,111,155897,249202.0,5.506660e+09,222315,Vasopressin,0.125
5,111,155897,249202.0,5.506661e+09,221906,Norepinephrine,0.200
6,111,155897,249202.0,5.506662e+09,221749,Phenylephrine,2.254
7,111,155897,249202.0,5.506664e+09,221906,Norepinephrine,0.250
8,111,155897,249202.0,5.506664e+09,222315,Vasopressin,0.200
9,111,155897,249202.0,5.506666e+09,221289,Epinephrine,0.020


## Demographics: Age, Gender, Weight

In [42]:
query = '''
with data as
(
    with sub_ids as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    ),
    age_gender as
    (
        SELECT
          p.subject_id, p.gender
          , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
              AS age
        FROM patients p
        INNER JOIN admissions a
        ON p.subject_id = a.subject_id
        GROUP BY p.subject_id, p.dob, p.gender
        ORDER BY p.subject_id
    )
    select sub_ids.subject_id, age_gender.age, age_gender.gender
    from sub_ids
    left join age_gender
    on sub_ids.subject_id = age_gender.subject_id
    order by sub_ids.subject_id
),
wt_tab as
(
    select pat.subject_id, min(ce.value) as weight
    from patients pat
    left join chartevents ce
    on pat.subject_id = ce.subject_id
    where ce.itemid in (580,581,226512)
    group by pat.subject_id
    order by pat.subject_id
)
select data.subject_id, data.age, data.gender, wt_tab.weight
from data
left join wt_tab
on data.subject_id = wt_tab.subject_id
order by data.subject_id
'''

data = pd.read_sql_query(query,con)
data.to_csv(os.path.join(export_dir,'demographics.csv'),index=False,sep=',')
data.head(n=15)

,subject_id,age,gender,weight
0,3,76.52,M,106
1,4,47.84,F,53.599998474121094
2,6,65.94,F,None
3,8,0.00,M,None
4,9,41.79,M,100.30000305175781
5,10,0.00,F,None
6,11,50.15,F,None
7,12,72.37,M,81
8,13,39.86,F,74.599998474121094
9,17,47.45,F,68


## Fluid input

### Data source : CareVue

In [49]:
query = """
with t1 as
(
    with sub_ids as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select sub_ids.subject_id, hadm_id, icustay_id, extract(epoch from charttime) as charttime, itemid, amount,
    case when itemid in (30176,30315) then amount *0.25
    when itemid in (30161) then amount *0.3
    when itemid in (30020,30321, 30015,225823,30186,30211,30353,42742,42244,225159,225159,225159) then amount *0.5
    when itemid in (227531) then amount *2.75
    when itemid in (30143,225161) then amount *3
    when itemid in (30009,220862) then amount *5
    when itemid in (30030,220995,227533) then amount *6.66
    when itemid in (228341) then amount *8
    else amount end as tev -- total equivalent volume
    from inputevents_cv
    -- only RT itemids
    left join sub_ids
    on sub_ids.subject_id = inputevents_cv.subject_id
    where amount is not null and itemid in (225158,225943,226089,225168,225828,225823,220862,220970,220864,225159,220995,
    225170,225825,227533,225161,227531,225171,225827,225941,225823,225825,225941,225825,228341,225827,30018,30021,30015,
    30296,30020,30066,30001,30030,30060,30005,30321,3000630061,30009,30179,30190,30143,30160,30008,30168,30186,30211,
    30353,30159,30007,30185,30063,30094,30352,30014,30011,30210,46493,45399,46516,40850,30176,30161,30381,30315,42742,
    30180,46087,41491,30004,42698,42244)
    order by sub_ids.subject_id, hadm_id, icustay_id, charttime, itemid
)


select subject_id, hadm_id, icustay_id, charttime, t1.itemid, d.label, round(cast(amount as numeric),3) as amount, 
round(cast(tev as numeric),3) as tev -- total equivalent volume
from t1
inner join d_items d
on t1.itemid = d.itemid

"""
d = pd.read_sql_query(query,con)
d.to_csv(export_dir+'fluid_cv.csv',index=False,sep=',')
d.head(n=15)

,subject_id,hadm_id,icustay_id,charttime,itemid,label,amount,tev
0,3.0,145834.0,211552.0,4.159296e+09,30018,.9% Normal Saline,10000.0,10000.0
1,3.0,145834.0,211552.0,4.159303e+09,30001,Packed RBC's,375.0,375.0
2,3.0,145834.0,211552.0,4.159310e+09,30018,.9% Normal Saline,0.5,0.5
3,3.0,145834.0,211552.0,4.159310e+09,30018,.9% Normal Saline,10.0,10.0
4,3.0,145834.0,211552.0,4.159310e+09,30018,.9% Normal Saline,125.0,125.0
5,3.0,145834.0,211552.0,4.159314e+09,30018,.9% Normal Saline,2.0,2.0
6,3.0,145834.0,211552.0,4.159321e+09,30018,.9% Normal Saline,4.0,4.0
7,3.0,145834.0,211552.0,4.159325e+09,30018,.9% Normal Saline,2.0,2.0
8,3.0,145834.0,211552.0,4.159328e+09,30018,.9% Normal Saline,2.0,2.0
9,3.0,145834.0,211552.0,4.159332e+09,30018,.9% Normal Saline,2.0,2.0


### Data source: MetaVision

In [50]:
query = """
with t1 as
(
    with sub_ids as 
    (
        select oe.subject_id, count(*)
        from outputevents oe
        inner join d_items d on oe.itemid = d.itemid
        where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
        227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
        group by oe.subject_id
    )
    select sub_ids.subject_id, hadm_id, icustay_id, extract(epoch from starttime) as starttime, 
    extract(epoch from endtime) as endtime, itemid, amount, rate,
    case when itemid in (30176,30315) then amount *0.25
    when itemid in (30161) then amount *0.3
    when itemid in (30020,30015,225823,30321,30186,30211, 30353,42742,42244,225159) then amount *0.5 --
    when itemid in (227531) then amount *2.75
    when itemid in (30143,225161) then amount *3
    when itemid in (30009,220862) then amount *5
    when itemid in (30030,220995,227533) then amount *6.66
    when itemid in (228341) then amount *8
    else amount end as tev -- total equivalent volume
    from inputevents_mv
    -- only real time items !!
    left join sub_ids
    on sub_ids.subject_id = inputevents_mv.subject_id
    where icustay_id is not null and amount is not null and itemid in (225158,225943,226089,225168,225828,225823,220862,
    220970,220864,225159,220995,225170,225825,227533,225161,227531,225171,225827,225941,225823,225825,225941,225825,
    228341,225827,30018,30021,30015,30296,30020,30066,30001,30030,30060,30005,30321,3000630061,30009,30179,30190,30143,
    30160,30008,30168,30186,30211,30353,30159,30007,30185,30063,30094,30352,30014,30011,30210,46493,45399,46516,40850,
    30176,30161,30381,30315,42742,30180,46087,41491,30004,42698,42244)
    order by sub_ids.subject_id, hadm_id, icustay_id, starttime, itemid
)


select subject_id, hadm_id, icustay_id, starttime, endtime, t1.itemid, d.label, round(cast(amount as numeric),3) as amount, 
round(cast(tev as numeric),3) as tev -- total equivalent volume
from t1
inner join d_items d
on t1.itemid = d.itemid

"""
d = pd.read_sql_query(query,con)
d.to_csv(export_dir+'fluid_mv.csv',index=False,sep=',')
d.head(n=15)

,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,label,amount,tev
0,23.0,124321,234044,5.926566e+09,5.926570e+09,225158,NaCl 0.9%,13.432,13.432
1,23.0,124321,234044,5.926570e+09,5.926572e+09,225158,NaCl 0.9%,5.346,5.346
2,23.0,124321,234044,5.926572e+09,5.926598e+09,225158,NaCl 0.9%,547.500,547.500
3,23.0,124321,234044,5.926572e+09,5.926582e+09,225158,NaCl 0.9%,17.339,17.339
4,23.0,124321,234044,5.926582e+09,5.926586e+09,225158,NaCl 0.9%,8.151,8.151
5,23.0,124321,234044,5.926586e+09,5.926598e+09,225158,NaCl 0.9%,13.295,13.295
6,23.0,124321,234044,5.926598e+09,5.926598e+09,225158,NaCl 0.9%,1.670,1.670
7,23.0,124321,234044,5.926598e+09,5.926615e+09,225158,NaCl 0.9%,452.500,452.500
8,23.0,124321,234044,5.926598e+09,5.926613e+09,225158,NaCl 0.9%,410.000,410.000
9,23.0,124321,234044,5.926599e+09,5.926599e+09,225158,NaCl 0.9%,250.000,250.000


## Sedative Medications

In [51]:
query = '''

with sub_ids as 
(
    select oe.subject_id, count(*)
    from outputevents oe
    inner join d_items d on oe.itemid = d.itemid
    where d.itemid in (40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,
    227510,226561,227489,226584,226563,226564,226565,226557,226558) and oe.valueuom is not NULL
    group by oe.subject_id
),
sedative_med as
(
    select subject_id, hadm_id, icustay_id, extract(epoch from startdate) as starttime, extract(epoch from enddate) as endtime,
    case 
        when lower(drug) like '%trazodone%' then 1
        when lower(drug) like '%hydroxyzine%' then 1
        when lower(drug) like '%ativan%' then 1
        when lower(drug) like '%lorazepam%' then 1
        when lower(drug) like '%promethazine%' then 1
        when lower(drug) like '%fentanyl%' then 1
        when lower(drug) like '%dexmedetomidine%' then 1
        when lower(drug) like '%phenobarbital%' then 1
        when lower(drug) like '%pentobarbital%' then 1
        when lower(drug) like '%chloral%hydrate%' then 1
        else 0 
    end as sedative
    from prescriptions
    order by subject_id, hadm_id, icustay_id, starttime
)
select sub_ids.subject_id, sed.hadm_id, sed.icustay_id, sed.starttime, sed.endtime, sed.sedative
from sub_ids
left join sedative_med sed
on sub_ids.subject_id = sed.subject_id
order by sub_ids.subject_id, sed.hadm_id, sed.icustay_id, sed.starttime

'''
d = pd.read_sql_query(query,con)
d.to_csv(export_dir+'sedative_med.csv',index=False,sep=',')
d.head(n=15)

,subject_id,hadm_id,icustay_id,starttime,endtime,sedative
0,3,NaN,NaN,NaN,NaN,NaN
1,4,185777.0,294638.0,6.980515e+09,6.980515e+09,0.0
2,4,185777.0,294638.0,6.980515e+09,6.980515e+09,0.0
3,4,185777.0,294638.0,6.980515e+09,6.981120e+09,0.0
4,4,185777.0,294638.0,6.980515e+09,6.980774e+09,0.0
5,4,185777.0,294638.0,6.980515e+09,6.980602e+09,0.0
6,4,185777.0,294638.0,6.980515e+09,6.981120e+09,0.0
7,4,185777.0,294638.0,6.980515e+09,6.980515e+09,0.0
8,4,185777.0,294638.0,6.980515e+09,6.980515e+09,0.0
9,4,185777.0,294638.0,6.980515e+09,6.980515e+09,0.0
